In [15]:
import pandas as pd
import numpy as np

import os

# Load metadata

In [16]:
nl = "\n"

In [17]:
metadata = pd.read_csv(
    os.path.join(
        os.path.abspath(""),
        "gutenberg",
        "SPGC_metadata_20180718.csv"
    ), 
    index_col="id"
)

In [18]:
metadata = metadata[
    (metadata["language"] == "['en']")
][["title", "author"]].dropna()

In [19]:
# Due to memory limitations, we only use the first 1000 books. 
metadata = metadata.iloc[:1000]

In [20]:
metadata

,title,author
id,,
PG1,The Declaration of Independence of the United ...,"Jefferson, Thomas"
PG2,The United States Bill of Rights: The Ten Orig...,United States
PG3,John F. Kennedy's Inaugural Address,"Kennedy, John F. (John Fitzgerald)"
PG4,Lincoln's Gettysburg Address: Given November 1...,"Lincoln, Abraham"
PG5,The United States Constitution,United States
...,...,...
PG1068,Personal Memoirs of U. S. Grant — Volume 2,"Grant, Ulysses S. (Ulysses Simpson)"
PG1069,Four Short Stories By Emile Zola,"Zola, Émile"
PG1074,The Sea-Wolf,"London, Jack"


# Load text

In [21]:
# Get all files in the directory and subdirectories that end with .txt
files = [
    os.path.join(root, name)
    for root, dirs, files in os.walk(
        os.path.join(
            os.path.abspath(""),
            "gutenberg",
            "SPGC_tokens_20180718"
        )
    )
    for name in files
    if name.endswith(".txt")
]

k = 5
print(
    f"Found {len(files)} files. {nl}"
    f"First {k} file(s): {nl}{files[:k]}"
)

Found 55905 files. 
First 5 file(s): 
['C:\\Users\\Aleks Lyubenov\\Documents\\[LMU] Classes\\[3] Winter 2023\\Data Security\\Project\\EmbeddingAttacks\\data\\gutenberg\\SPGC_tokens_20180718\\PG10000_tokens.txt', 'C:\\Users\\Aleks Lyubenov\\Documents\\[LMU] Classes\\[3] Winter 2023\\Data Security\\Project\\EmbeddingAttacks\\data\\gutenberg\\SPGC_tokens_20180718\\PG10001_tokens.txt', 'C:\\Users\\Aleks Lyubenov\\Documents\\[LMU] Classes\\[3] Winter 2023\\Data Security\\Project\\EmbeddingAttacks\\data\\gutenberg\\SPGC_tokens_20180718\\PG10002_tokens.txt', 'C:\\Users\\Aleks Lyubenov\\Documents\\[LMU] Classes\\[3] Winter 2023\\Data Security\\Project\\EmbeddingAttacks\\data\\gutenberg\\SPGC_tokens_20180718\\PG10003_tokens.txt', 'C:\\Users\\Aleks Lyubenov\\Documents\\[LMU] Classes\\[3] Winter 2023\\Data Security\\Project\\EmbeddingAttacks\\data\\gutenberg\\SPGC_tokens_20180718\\PG10004_tokens.txt']


In [27]:
texts = pd.DataFrame(
    [
        [os.path.basename(file).split(".")[0], open(file, "r", encoding="utf8").read().split("\n")]
        for file in files
        if os.path.basename(file).split(".")[0].replace("_tokens", "") in metadata.index
    ],
    columns=["id", "text"]
)

texts["id"] = texts["id"].apply(lambda x: x.replace("_tokens", ""))
texts = texts.set_index("id")

# Add author to text. 
texts = texts.join(metadata["author"], how="inner")

In [28]:
texts

,text,author
id,,
PG1001,"[the, divine, comedy, of, dante, alighieri, tr...",Dante Alighieri
PG1002,"[the, divine, comedy, of, dante, alighieri, tr...",Dante Alighieri
PG1003,"[the, divine, comedy, of, dante, alighieri, tr...",Dante Alighieri
PG1004,"[the, divine, comedy, of, dante, alighieri, tr...",Dante Alighieri
PG1005,"[hell, or, the, inferno, from, the, divine, co...",Dante Alighieri
...,...,...
PG994,"[riders, to, the, sea, a, play, in, one, act, ...","Synge, J. M. (John Millington)"
PG995,"[ballads, of, a, bohemian, by, robert, service...","Service, Robert W. (Robert William)"
PG996,"[don, quixote, complete, by, miguel, de, cerva...","Cervantes Saavedra, Miguel de"


# Embeddings

In [24]:
# TODO: Embed